In [1]:
from utils import *
import numpy as np
import matplotlib.pyplot as plt
import pyqpanda as pq
import pyvqnet as pv
import random

%matplotlib inline

In [2]:
train_data = Dataset(
    "data/cifar-10-batches-py/data_batch_1",
    "data/cifar-10-batches-py/data_batch_2",
    "data/cifar-10-batches-py/data_batch_3",
    "data/cifar-10-batches-py/data_batch_4",
    "data/cifar-10-batches-py/data_batch_5",
)
train_data.enhance()
test_data = Dataset("data/cifar-10-batches-py/test_batch")

In [3]:
def pqctest(input, param, qubits, cbits, machine):
    n = int(np.sqrt(len(input)))
    prog = pq.QProg()
    for i in range(n):
        prog.insert(pq.U3(qubits[i], param[i], param[i + n], param[i + 2 * n]))
    for i in range(n):
        for j in range(n):
            t = i * n + j
            if i == j:
                prog.insert(pq.P(qubits[i], np.pi * input[t]))
            else:
                prog.insert(pq.CR(qubits[i], qubits[j], np.pi * input[t]))
    for i in range(n):
        prog.insert(
            pq.U3(qubits[i], param[i + 3 * n], param[i + 4 * n], param[i + 5 * n])
        )
    prob = pv.qnn.measure.ProbsMeasure([i for i in range(n)], prog, machine, qubits)
    re = [0] * n
    for i in range(n):
        re[i] = prob[1 << i]
    return re


class Model(pv.nn.module.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv = pv.nn.Conv2D(3, 1, (3, 3), (1, 1))
        self.pool = pv.nn.MaxPool2D((3, 3), (3, 3))
        self.ac = pv.nn.Sigmoid()
        self.qc = pv.qnn.quantumlayer.QuantumLayer(
            pqctest, 6 * 10, "CPU", 10, diff_method="finite_diff"
        )

    def forward(self, x):
        x = self.conv(x)
        x = self.pool(x)
        x = pv.tensor.flatten(x, 1)
        x = self.ac(x)
        x = self.qc(x)
        return x

In [4]:
epoch = 100
batch = 16
holdout = 10000

model = Model()
print(sum(p.numel() for p in model.parameters()))

88


In [ ]:
X, Y = train_data.getdatas()
los = pv.nn.loss.CrossEntropyLoss()
opt = pv.optim.Adam(model.parameters())

acc_train = []
acc_test = []

for e in range(1, epoch + 1):
    model.train()

    correct_tot = 0
    for I, (x, y) in enumerate(
        pv.data.data_generator(
            X[:-holdout], Y[:-holdout], batch_size=batch, shuffle=True
        )
    ):
        i = I + 1
        # print(i)
        opt.zero_grad()
        y_pred = model(x)
        loss = los(y, y_pred)
        correct = sum(y_pred.argmax(1, False).to_numpy() == y)
        acc = correct / y.shape[0]
        correct_tot += correct
        loss.backward()
        opt._step()
        if i % 1 == 0:
            print(
                f"epoch {e}/{epoch} \t batch {batch*i}/{Y.shape[0]-holdout} \t loss {loss.item():.2f} \t accuracy {acc:.2f}"
            )

    model.eval()

    acc = correct_tot / (Y.shape[0] - holdout)
    print(f"********** train: epoch {e}/{epoch} \t accuracy {100*acc:.2f}% **********")
    acc_train.append(acc)

    correct_tot = 0
    for x, y in pv.data.data_generator(
        X[-holdout:], Y[-holdout:], batch_size=batch, shuffle=False
    ):
        y_pred = model(x)
        correct_tot += sum(y_pred.argmax(1, False).to_numpy() == y)

    acc = correct_tot / holdout
    print(f"********** test: epoch {e}/{epoch} \t accuracy {100*acc:.2f}% **********")
    acc_test.append(acc)

    # opt = pv.optim.SGD(model.parameters(), 0.01 * (1 - acc))

epoch 1/100 	 batch 16/90000 	 loss 2.30 	 accuracy 0.06
epoch 1/100 	 batch 32/90000 	 loss 2.30 	 accuracy 0.12
epoch 1/100 	 batch 48/90000 	 loss 2.30 	 accuracy 0.00
epoch 1/100 	 batch 64/90000 	 loss 2.30 	 accuracy 0.12
epoch 1/100 	 batch 80/90000 	 loss 2.30 	 accuracy 0.00
epoch 1/100 	 batch 96/90000 	 loss 2.30 	 accuracy 0.00
epoch 1/100 	 batch 112/90000 	 loss 2.30 	 accuracy 0.06
epoch 1/100 	 batch 128/90000 	 loss 2.30 	 accuracy 0.00
epoch 1/100 	 batch 144/90000 	 loss 2.30 	 accuracy 0.06
epoch 1/100 	 batch 160/90000 	 loss 2.30 	 accuracy 0.12
epoch 1/100 	 batch 176/90000 	 loss 2.30 	 accuracy 0.12
epoch 1/100 	 batch 192/90000 	 loss 2.30 	 accuracy 0.12
epoch 1/100 	 batch 208/90000 	 loss 2.30 	 accuracy 0.12
epoch 1/100 	 batch 224/90000 	 loss 2.30 	 accuracy 0.06
epoch 1/100 	 batch 240/90000 	 loss 2.30 	 accuracy 0.12
epoch 1/100 	 batch 256/90000 	 loss 2.30 	 accuracy 0.06
epoch 1/100 	 batch 272/90000 	 loss 2.30 	 accuracy 0.00
epoch 1/100 	 batch 

In [ ]:
pv.utils.storage.save_parameters(model.state_dict(), "train10.model")

In [ ]:
m = Model()
m.load_state_dict(pv.utils.storage.load_parameters("train10.model"))

In [ ]:
m.eval()
X, Y = test_data.getdatas()
correct_tot = 0
for x, y in pv.data.data_generator(X, Y, batch_size=batch, shuffle=False):
    y_pred = model(x)
    correct_tot += sum(y_pred.argmax(1, False).to_numpy() == y)
acc = correct_tot / Y.shape[0]
print(f"********** eval: accuracy {100*acc:.2f}% **********")